In [1]:
import pandas as pd

In [2]:
sample = pd.read_csv('sample_submission.csv')

In [3]:
sample.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [5]:
stores = pd.read_csv('stores.csv')

In [6]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [7]:
transactions = pd.read_csv('transactions.csv')

In [8]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [9]:
train = pd.read_csv('train.csv')

In [10]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


## EDA for stores.csv

In [11]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [12]:
len(stores)

54

In [17]:
len(stores['city'].value_counts())

22

In [19]:
len(stores['type'].value_counts())

5

In [20]:
stores['type'].value_counts()

D    18
C    15
A     9
B     8
E     4
Name: type, dtype: int64

In [29]:
stores[stores['type']=='D'].value_counts().to_frame().sort_values(by='city', ascending=False).head()

,,,,,0
store_nbr,city,state,type,cluster,
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,1
25,Salinas,Santa Elena,D,1,1
1,Quito,Pichincha,D,13,1
2,Quito,Pichincha,D,13,1
3,Quito,Pichincha,D,8,1


In [35]:
(len(stores[stores['type']=='D']) + len(stores[stores['type']=='C'] ))/len(stores)

0.6111111111111112

In [18]:
len(stores['cluster'].value_counts())

17

## Takeaways:
+ Total of 54 stores across 22 cities and 17 clusters
+ 5 store types, majority are D (61.1% of stores are type D or C)

## EDA for transactions.csv

In [36]:
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [37]:
len(transactions)

83488

In [43]:
transactions.dtypes

date            object
store_nbr        int64
transactions     int64
dtype: object

In [39]:
transactions['transactions'].value_counts()

1207    90
1200    86
1304    81
1282    80
1171    80
        ..
4646     1
4520     1
4220     1
4590     1
4392     1
Name: transactions, Length: 4993, dtype: int64

In [44]:
transactions[transactions['transactions']<=5].value_counts()

date        store_nbr  transactions
2016-09-07  43         5               1
dtype: int64

## Summary for transactions
+ Total of 83,488 values for transactions across the 54 stores

In [38]:
transactions.tail()

,date,store_nbr,transactions
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932
83487,2017-08-15,54,802


In [45]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np

In [46]:
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv')
train = pd.read_csv('train.csv')

In [47]:
# merge the data
merged_train = pd.merge(train, stores, on='store_nbr', how='left')
merged_train = pd.merge(merged_train, transactions, on=['store_nbr', 'date'], how='left')


In [51]:
# simple feature engineering
merged_train['date'] = pd.to_datetime(merged_train['date'])
merged_train['day_of_week'] = merged_train['date'].dt.day

In [67]:
# traun/test split
train_data = merged_train[merged_train['date'] < '2016-09-09']
test_data = merged_train[merged_train['date'] >= '2016-09-09']


In [87]:
train_data = train_data.dropna()
test_data = test_data.dropna()
train_data.isna().sum(), test_data.isna().sum()

(id              0
 date            0
 store_nbr       0
 family          0
 sales           0
 onpromotion     0
 city            0
 state           0
 type            0
 cluster         0
 transactions    0
 day_of_weel     0
 day_of_week     0
 dtype: int64,
 id              0
 date            0
 store_nbr       0
 family          0
 sales           0
 onpromotion     0
 city            0
 state           0
 type            0
 cluster         0
 transactions    0
 day_of_weel     0
 day_of_week     0
 dtype: int64)

In [88]:
# model building and evaluation
X_train = train_data[['day_of_week', 'transactions']]
y_train = np.log1p(train_data['sales'])

X_test = test_data[['day_of_week', 'transactions']]
y_test = np.log1p(test_data['sales'])

In [89]:
model = RandomForestRegressor(n_estimators=50)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(f"RMSLE: {rmsle}")

RMSLE: 0.6955553155754807


In [57]:
merged_train

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions,day_of_weel,day_of_week
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,NaN,1,1
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,NaN,1,1
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,NaN,1,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,NaN,1,1
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,2155.0,15,15
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,2155.0,15,15
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,2155.0,15,15
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,2155.0,15,15


In [62]:
(len(merged_train['date'].unique()))*.8

1347.2

In [66]:
# 1684 days starting with '2013-01-01'
from datetime import datetime, timedelta
day_1 = datetime.strptime('2013-01-01','%Y-%m-%d')
cut_off_date = day_1 + timedelta(1347)
cut_off_date

datetime.datetime(2016, 9, 9, 0, 0)

In [72]:
(len(X_train) - len(X_train.dropna()))/len(X_train) * 100

8.190375648807953